### Sorting, abstract class

In [41]:
import numpy as np
import functools
import copy
import operator


class Sorting(object):
    """排序算法抽象类
    
    - 通用功能：交换和移动
        + swap()
        + move()
    - 数据生成： random, almost sorted, reversed, few unique
    - 性能分析： time: #swap, #move; space: 
    """
    
    def __init__(self):
        self.L_random = None
        self.L_sorted = None
        self.L_reversed = None
        self.L_few_unique = None
        # self.regenerate_L() # 暴露给用户
        
        self.L_customized = None
        # self.customize_L()  # 暴露给用户
        
        # run & performance info
        self.origin_L = None
        self.L = None
        self.N = None
        
        self.count_call_dict = {}
        
        # decorator raw syntex
        self.alg_cmp = self.count_call(self.alg_cmp)
        self.swap = self.count_call(self.swap)
        self.copy_to = self.count_call(self.copy_to)
        
    def regenerate_L(self, N=30, few_unique_nvalue=5, seed=123456):
        np.random.seed(seed)
        self.L_random = list(np.random.choice(range(N), size=N, replace=False))
        self.L_sorted = list(range(N))
        self.L_reversed = list(range(N, 0, -1))
        self.L_few_unique = list(np.random.choice(range(few_unique_nvalue), size=N, replace=True))    
        
    def customize_L(self, L):
        self.L_customized = L 
    
    # ------------ sort -----------------------------
    def sort(self, L):
        if True:
            self.origin_L = copy.deepcopy(L)
            self.L = L
            self.N = len(L)
            
        self._sort(self.L)
        
        if True:
            Sorting.check_sorted(self.L)
            self.perfromance_summary()
        
    def _sort(self, L):
        raise NotImplementedError
        
    # ------------ utils && performance analysis -----------------------------    
    def count_call(self, func):
        self.count_call_dict[func.__name__] = 0
        @functools.wraps(func)
        def wrapper(*args, **kw):
            self.count_call_dict[func.__name__] += 1
            return func(*args, **kw)
        return wrapper
    
    def alg_cmp(self, a, op, b):
        return op(a, b)
    
    def swap(self, L, i, j):
        L[i], L[j] = L[j], L[i]
        
    def copy_to(self, ref, dest):
        dest = copy.deepcopy(ref)
            
    def perfromance_summary(self):
        print(self.origin_L)
        print('inversion number:', Sorting.get_inversion_number(self.origin_L))
        print(self.L, Sorting.check_sorted(self.L))
        print('------------------------')
        print('N =', self.N)
        print('========= Theretical Analysis =========')
        for (txt, f) in self.theretical_complexity():
            print(txt, '-->', str(f(self.N)))
        print('------------------------')
        print('time complexity: ', self.count_call_dict)
        print('space complexity:', )
        
    def theretical_complexity(self):
        raise NotImplementedError
        
    @staticmethod
    def get_inversion_number(L):
        n = len(L)
        return sum(L[i] > L[j] for i in range(n) for j in range(i+1, n))
    
    @staticmethod
    def check_sorted(L):
        return all(L[i] <= L[i+1] for i in range(len(L) - 1))

In [42]:
m = Sorting()
m.regenerate_L(N=20)
# m.sort(m.L_random)
# m.perfromance_summary()

### Bubble Sort

In [43]:
class BubbleSort(Sorting):
    """冒泡排序"""
    
    def _sort(self, L):
        n = len(L)
        for i in range(n):
            for j in range(n-1, i, -1):
                if self.alg_cmp(L[j-1], operator.__gt__, L[j]):
                    self.swap(L, j-1, j)
                    
    def theretical_complexity(self):
        return [
            ('#cmp: N(N-1)/2', lambda N: N * (N-1) / 2),
            ('#swap  bad-case reversed: N(N-1)/2', lambda N: N * (N-1) / 2),
            ('#swap  good-case sorted: 0', lambda N: 0),
            ('#space: 0', lambda N: 0)
        ]

In [44]:
m_bb = BubbleSort()
m_bb.regenerate_L(N=20)

m_bb.sort(L=m.L_random)

[19, 13, 12, 9, 5, 6, 2, 3, 18, 8, 4, 7, 0, 14, 16, 15, 11, 17, 10, 1]
inversion number: 101
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19] True
------------------------
N = 20
========= Theretical Analysis =========
#cmp: N(N-1)/2 --> 190.0
#swap  bad-case reversed: N(N-1)/2 --> 190.0
#swap  good-case sorted: 0 --> 0
#space: 0 --> 0
------------------------
time complexity:  {'alg_cmp': 190, 'swap': 101, 'copy_to': 0}
space complexity:


In [46]:
m_bb2 = BubbleSort()
m_bb2.regenerate_L(N=10)

m_bb2.sort(L=m_bb2.L_random)

[8, 7, 6, 4, 5, 3, 0, 9, 2, 1]
inversion number: 35
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9] True
------------------------
N = 10
========= Theretical Analysis =========
#cmp: N(N-1)/2 --> 45.0
#swap  bad-case reversed: N(N-1)/2 --> 45.0
#swap  good-case sorted: 0 --> 0
#space: 0 --> 0
------------------------
time complexity:  {'alg_cmp': 45, 'swap': 35, 'copy_to': 0}
space complexity:


## Python笔记：关于 funcion 和 bound method

> https://stackoverflow.com/questions/7891277/why-does-setattr-fail-on-a-bound-method


### 遇到的问题：试图对成员函数setattr时会抛出错误

AttributeError: 'method' object has no attribute 'foo'

### 原理：function 与 bound method 的本质是什么？

function object 是某种“真实存在的”对象，可以设置任意attr，它有一个`__dict__`属性来存储所有的attr    

而与特定实例绑定的 bound method 不同，它没有具体的函数实体。

class中的def会创建一个（属于cls命名空间中的）funcion object，通过`cls.m`获得。   
而与具体实例绑定的 bound method 不存储函数实体，它只是是记录两个对象的引用：

- 其宿主 `instance.__self__`
- 定义在类中的函数实体 `instance.m.__func__` (is `cls.m`)

当bound method 被调用时，实际执行的是`cls.m(instance.__self__, ...)`   
该类所有实例的method，其函数实体`instance.m.__func__` 都指向同一个对象，即类创建时创建的 function object `cls.m`

> 回忆quantx是怎么在R语言中实现“类”的？

### 回到刚才的问题

commit e785ee8 实际上是将属性设置在了cls.m上（定义在class namespace中，是函数对象），是错误的。

```
def dec(func):
    def wrapper(*arg, **kw):
        func.xx = xx              # ✔️ 不会报错，但注意，属性是设置在了 KLS.func上
        return func(*arg, **kw)
    return wrapper

class KLS:
    @dec
    def foo:
        setattr(self.bar, 'xx', xx)   # ❌ bounded method 不是函数对象，不可以设置属性
        setattr(self.foo, 'xx, xx)  # ❌原因同上

ins = KLS()
ins.foo.xx = xx   # ❌ 原因同上
```

In [26]:
m_bb.swap is m_bb2.swap   # different object, since it is bound method of two different istance

False

In [24]:
m_bb.swap.__func__ is m_bb2.swap.__func__  # same object ...

True

In [29]:
m_bb.swap.__func__ is BubbleSort.swap    # ... since they are all ref to cls.funcion

True

In [31]:
m_bb.swap is BubbleSort.swap   # bound method and cls.funcion is not same thing !

False

In [18]:
BubbleSort.swap  # it is function object !

<function __main__.Sorting.swap(self, L, i, j)>

In [19]:
m_bb.swap  # it is bound method

<bound method Sorting.swap of <__main__.BubbleSort object at 0x1057068d0>>

In [20]:
m_bb2.swap  # bound method of another istance

<bound method Sorting.swap of <__main__.BubbleSort object at 0x105706390>>

### 那么怎么实现“成员方法调用次数计数”这个需求呢？我自己想的一个实现：decorator最原始的语法

> https://medium.com/@vadimpushtaev/decorator-inside-python-class-1e74d23107f6

1. 直接在成员方法的`def`上加`@dec`并不work
2. staticmethod也不行，因为 @staticmethod 对于inside class / outside class 的behavior不同

In [63]:
class A(object):
    def __init__(self):
        print('calling __init__')
        self.count_call_dict = {}
        
        self.a = self.count_call(self.a)
        self.b = self.count_call(self.b)
        
    def a(self):
        print('aaaaaaaaaaa')
    
    def b(self):
        print('bbbbbbbbbbb')
        
    def count_call(self, func):
        print('Calling count_call')
        self.count_call_dict[func.__name__] = 0
        def wrapper(*args, **kw):
            print('func name is', func.__name__)
            self.count_call_dict[func.__name__] += 1
            return func(*args, **kw)
        return wrapper

In [69]:
foooo = A()
b___ = A()

foooo.a(); foooo.a(); foooo.a()   # foooo.a = 3
foooo.b(); foooo.b()              # foooo.b = 2
b___.a()
foooo.a()                         # foooo.a = 4
b___.b(); b___.b()

print(foooo.count_call_dict)
print(b___.count_call_dict)

calling __init__
Calling count_call
Calling count_call
calling __init__
Calling count_call
Calling count_call
func name is a
aaaaaaaaaaa
func name is a
aaaaaaaaaaa
func name is a
aaaaaaaaaaa
func name is b
bbbbbbbbbbb
func name is b
bbbbbbbbbbb
func name is a
aaaaaaaaaaa
func name is a
aaaaaaaaaaa
func name is b
bbbbbbbbbbb
func name is b
bbbbbbbbbbb
{'a': 4, 'b': 2}
{'a': 1, 'b': 2}


In [67]:




bar.count_call_dict

calling __init__
Calling count_call
Calling count_call
func name is a
aaaaaaaaaaa
func name is b
bbbbbbbbbbb


{'a': 1, 'b': 1}